<a href="https://colab.research.google.com/github/Inteli-College/2024-2A-T05-CC09-G04/blob/detectron2/yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Clonar o repositório YOLOv5
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

# Instalar as dependências
!pip install -r requirements.txt


In [ ]:
!ls hubconf.py


hubconf.py


Este código carrega imagens e suas respectivas anotações (bounding boxes) em um diretório e desenha essas caixas sobre as imagens. Ele converte coordenadas normalizadas para pixels e exibe as imagens com as bounding boxes usando **OpenCV** e **Matplotlib**.

In [ ]:
'''
import os
import cv2
import matplotlib.pyplot as plt

# Diretório das imagens e anotações
image_dir = '/content/drive/MyDrive/dateset/images'
annotation_dir = '/content/drive/MyDrive/dateset/labels'

# Função para desenhar bounding boxes nas imagens
def draw_bounding_boxes(image, boxes):
    h, w = image.shape[:2]
    for box in boxes:
        class_id, x_center, y_center, width, height = box
        # Converta as coordenadas normalizadas para pixels
        x_center, y_center, width, height = x_center * w, y_center * h, width * w, height * h
        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        x_max = int(x_center + width / 2)
        y_max = int(y_center + height / 2)

        # Desenhar a bounding box na imagem
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)

    return image

# Função para carregar e verificar imagens e anotações
def load_and_display(image_path, annotation_path):
    # Carregar a imagem
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converter para RGB para exibição correta no matplotlib

    # Carregar as anotações
    boxes = []
    with open(annotation_path, 'r') as f:
        for line in f.readlines():
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            boxes.append([class_id, x_center, y_center, width, height])

    # Desenhar as bounding boxes na imagem
    image_with_boxes = draw_bounding_boxes(image_rgb, boxes)

    # Mostrar a imagem com bounding boxes
    plt.imshow(image_with_boxes)
    plt.axis('off')
    plt.show()

# Listar os arquivos de imagem no diretório
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]

# Testar as primeiras 5 imagens e suas anotações
for image_file in image_files[:5]:
    image_path = os.path.join(image_dir, image_file)
    annotation_file = image_file.replace('.png', '.txt')
    annotation_path = os.path.join(annotation_dir, annotation_file)

    if os.path.exists(annotation_path):
        print(f"Mostrando imagem e anotações para {image_file}")
        load_and_display(image_path, annotation_path)
    else:
        print(f"Anotação não encontrada para {image_file}")


Este código divide um conjunto de imagens em diretórios de **treino** e **validação** (80/20) e move os arquivos correspondentes para as pastas corretas. As anotações associadas às imagens também são movidas. Ele garante que cada imagem tenha uma anotação correspondente e avisa se alguma estiver faltando.

In [ ]:
'''
import os
import random
import shutil

# Diretórios originais
image_dir = '/content/drive/MyDrive/dateset/images'
label_dir = '/content/drive/MyDrive/dateset/labels'

# Diretórios de destino
train_images_dir = '/content/drive/MyDrive/cattle_dataset/dateset/images/train'
val_images_dir = '/content/drive/MyDrive/cattle_dataset/dateset/images/val'
train_labels_dir = '/content/drive/MyDrive/cattle_dataset/dateset/labels/train'
val_labels_dir = '/content/drive/MyDrive/cattle_dataset/dateset/labels/val'

# Criar diretórios de treino e validação
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Listar todos os arquivos de imagem
images = [f for f in os.listdir(image_dir) if f.endswith('.png') or f.endswith('.jpg')]

# Embaralhar as imagens
random.shuffle(images)

# Dividir 80% para treino e 20% para validação
split_idx = int(0.8 * len(images))
train_images = images[:split_idx]
val_images = images[split_idx:]

# Função para mover arquivos
def move_files(image_list, target_image_dir, target_label_dir):
    for image in image_list:
        image_path = os.path.join(image_dir, image)
        label_path = os.path.join(label_dir, image.replace('.png', '.txt').replace('.jpg', '.txt'))

        if os.path.exists(label_path):
            shutil.move(image_path, os.path.join(target_image_dir, image))
            shutil.move(label_path, os.path.join(target_label_dir, os.path.basename(label_path)))
        else:
            print(f"Anotação não encontrada para {image}")

# Mover os arquivos para treino e validação
move_files(train_images, train_images_dir, train_labels_dir)
move_files(val_images, val_images_dir, val_labels_dir)

print("Divisão de treino/validação completa!")


Divisão de treino/validação completa!


Este comando treina o modelo **YOLOv5m** usando imagens de **128x128**, com batch size de **16** por **100 epochs**. O arquivo **data.yaml** define o dataset, e o modelo pré-treinado **yolov5m.pt** é usado como base. A opção `--cache` acelera o treinamento carregando os dados na memória.

In [ ]:
!python train.py --img 128 --batch 16 --epochs 100 --data /content/drive/MyDrive/dateset/data.yaml --weights yolov5m.pt --cache

In [ ]:
import os

# Verifique o diretório principal onde os resultados de treinamento estão
print(os.listdir('runs/train'))

# Verifique o conteúdo do diretório do experimento (substitua 'exp' por 'exp1', 'exp2', etc. se necessário)
print(os.listdir('runs/train/exp2'))

# Verifique o conteúdo do diretório de pesos para encontrar o arquivo 'best.pt'
print(os.listdir('runs/train/exp2/weights'))


In [ ]:
from IPython.display import Image
Image(filename='runs/train/exp2/results.png', width=600)


Este comando realiza a **inferência** com o modelo treinado **best.pt** em uma imagem específica de **640x640 pixels**. A imagem de origem está no diretório especificado, e o modelo só detectará objetos com confiança maior que **0.4**.

In [ ]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --source /content/drive/MyDrive/frames-teste/frames-teste/06-08-24_video1_imagem_termica_edit_26965.png --conf 0.4
